In [1]:
import xarray as xr
import rioxarray
from rasterio.enums import Resampling
import numpy as np
import os
from tqdm import tqdm
from colorama import Fore, Back, Style
import math

In [2]:
def check_and_assign_crs(indices_nc_path, koppen_tif_path):
    
    # Load indices and Koppen map data
    indices_data = xr.open_dataset(indices_nc_path)
    koppen_data = rioxarray.open_rasterio(koppen_tif_path)
    
    # Get CRS of Koppen map
    koppen_crs = koppen_data.rio.crs
    
    # Assign CRS to indices data if not present
    if not indices_data.rio.crs:
        indices_data = indices_data.rio.write_crs(koppen_crs)
    
    return indices_data, koppen_data

In [3]:
def convert_longitudes(indices_data):
    # Convert longitudes from 0-360 to -180 to 180 if necessary
    if indices_data.lon.max() > 180:
        indices_data['lon'] = ((indices_data['lon'] + 180) % 360) - 180
        indices_data = indices_data.sortby(indices_data.lon)
    return indices_data

In [4]:
def set_spatial_dims(indices_data):
    # Set spatial dimensions
    indices_data = indices_data.rename({'lon': 'x', 'lat': 'y'})
    return indices_data

In [5]:
def resample_to_match_resolution(indices_data, koppen_data, output_nc_path):
    # Reproject indices data to match Koppen map resolution and grid
    resampled_indices = indices_data.rio.reproject_match(
        koppen_data,
        resampling=Resampling.bilinear  # Using bilinear interpolation
    )
    # Save the resampled indices data
    resampled_indices.to_netcdf(output_nc_path)

In [6]:
# Define paths to your data
koppen_tif_path = '../Raw_Data/koppen_geiger_0p5.tif'

# Loop through all models and indices
models_dir = '../Processed_Data/HTI_Indices/'
output_dir = '../Processed_Data/HTI_Indices_Resampled/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [7]:
for model_name in os.listdir(models_dir):
    model_path = os.path.join(models_dir, model_name)
    
    if os.path.isdir(model_path):
        
        for scenario in os.listdir(model_path):
            scenario_path = os.path.join(model_path, scenario)
            
            if os.path.isdir(scenario_path):
                
                for index_name in tqdm(os.listdir(scenario_path),'Processing Varibles'):
                    index_path = os.path.join(scenario_path, index_name)
                    if index_path.endswith('.nc'):
                        index_data, koppen_data = check_and_assign_crs(index_path, koppen_tif_path)
                        index_data = convert_longitudes(index_data)
                        index_data = set_spatial_dims(index_data)
                        
                        output_HTI_Index_Resampled_nc_path = os.path.join(output_dir,model_name,scenario,f'resampled_{index_name}')
                        os.makedirs(os.path.dirname(output_HTI_Index_Resampled_nc_path), exist_ok=True)
                        resample_to_match_resolution(index_data, koppen_data, output_HTI_Index_Resampled_nc_path)

Processing Varibles: 100%|█████████████████████████████████████████████████████████████| 12/12 [05:25<00:00, 27.16s/it]
